## Dresslar CAS522 M6 Notebook

Consider the dynamics of the following Boolean network:

<br />
<img src="../public/Assignment Module 6-1.png" width="400px"/>
<br />

### Question 1
> Determine the global dynamics of the network by finding all cycles and fix-point attractors. Determine the sizes of all basins of attraction. Perform the analysis without using any numerical code.


### Question 2
> Then double-check your results in python, and submit your notebook. 
